In [1]:
import pandas as pd
import numpy as np
import re

In [4]:
presenciais = pd.read_csv('componentes-curriculares-presenciais.csv', delimiter=';')
semi = pd.read_csv('componentes-curriculares-semi-presenciais.csv', delimiter=';')
ead = pd.read_csv('componentes-curriculares-ead.csv', delimiter=';')
componentes = presenciais.append(semi).append(ead)
componentes = componentes[componentes.nivel=='G']

In [15]:
cursos = pd.read_csv('cursos-graduacao.csv', delimiter=';')

In [19]:
matriculas_20171 = pd.read_csv('matricula-componente-20171.csv', delimiter=';')
matriculas_20172 = pd.read_csv('matricula-componente-20172.csv', delimiter=';')
matriculas = matriculas_20171.append(matriculas_20172)
matriculas = matriculas.groupby('id_turma', as_index=False).first()
matriculas = matriculas[matriculas.id_curso.isin(cursos.id_curso)]

In [23]:
turmas_20171 = pd.read_csv('turmas-2017.1.csv', delimiter=';')
turmas_20172 = pd.read_csv('turmas-2017.2.csv', delimiter=';')
turmas = turmas_20171.append(turmas_20172)
turmas = turmas.groupby('id_turma', as_index=False).first()

In [36]:
matriculas['id_componente_curricular'] = matriculas.apply(lambda x: turmas[turmas.id_turma==x.id_turma].id_componente_curricular.values[0], axis=1)

In [39]:
matriculas['nome_componente_curricular'] = matriculas.apply(lambda x: componentes[componentes.id_componente==x.id_componente_curricular].nome.values[0], axis=1)

In [44]:
disciplinas_cursos = matriculas[['id_curso', 'nome_componente_curricular']].groupby('id_curso').apply(lambda x: ' '.join(x.nome_componente_curricular))

In [93]:
from unicodedata import normalize
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII','ignore').decode('ASCII')

    if __name__ == '__main__':
        from doctest import testmod
        testmod()

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopw = set(stopwords.words('portuguese'))

def palavras_por_curso (curso, n_palavras):
    disciplinas = disciplinas_cursos[cursos[cursos.nome==curso].id_curso].values
    disciplinas = ' '.join(disciplinas)
    words = re.sub('[!@#$%^&*()\[\]{};:,.\-/<>\'?\|`=_+]', "", str(remover_acentos(disciplinas))).lower().split()
    words = [w for w in words if w not in stopw]
    return pd.Series(words).value_counts()[:n_palavras].index

## Lista de Cursos

In [87]:
cursos.nome

0                                     ADMINISTRAÇÃO
1                                     ADMINISTRAÇÃO
2                             ADMINISTRAÇÃO PÚBLICA
3                                         AGRONOMIA
4             ANÁLISE E DESENVOLVIMENTO DE SISTEMAS
5                           ARQUITETURA E URBANISMO
6                                     ARTES VISUAIS
7                                   BIBLIOTECONOMIA
8                                       BIOMEDICINA
9                             CIÊNCIA DA COMPUTAÇÃO
10                               CIÊNCIAS ATUARIAIS
11                              CIÊNCIAS BIOLÓGICAS
12                              CIÊNCIAS BIOLÓGICAS
13                              CIÊNCIAS BIOLÓGICAS
14                               CIÊNCIAS CONTÁBEIS
15                               CIÊNCIAS CONTÁBEIS
16                              CIÊNCIAS ECONÔMICAS
17                            CIÊNCIAS E TECNOLOGIA
18                                 CIÊNCIAS SOCIAIS
19          

In [98]:
curso = 'DESIGN'
n_palavras = 5
palavras_por_curso (curso, n_palavras)

Index(['desenho', 'projeto', 'produto', 'visual', 'programacao'], dtype='object')